 Alocação de salas para disciplinas (timetabling simplificado)

In [4]:
"""Descrição:
Precisa alocar turmas em salas e horários com algumas restrições simples:
• Conflitos de professor (não pode dar duas aulas ao mesmo tempo).
• Capacidade da sala ≥ número de alunos.
Não precisa ser um timetabling completo – um caso com poucas turmas já gera um bom
espaço de estados.
• Estado: conjunto de disciplinas já alocadas + configuração atual (quais horários
de quais salas estão ocupados).
• Ações: escolher uma disciplina ainda não alocada e tentar colocá-la num (sala,
horário) livre.
• Objetivo: todas as disciplinas alocadas sem violar restrições (e opcionalmente
minimizar “sobra” de horários, uso de salas grandes demais, etc.).
Heurísticas (A* / gulosa):
• Número de disciplinas restantes.
• Soma de “penalidades” por disciplinas “difíceis” (muitas restrições) ainda não
alocadas.
• Grau de conflito (disciplinas que compartilham professores ou estudantes) como
prioridade.
BFS/DFS híbridos:
• DFS com backtracking básico (clássico de CSP).
• Estratégia híbrida: BFS nas primeiras alocações (para garantir diversidade de
soluções), depois DFS profunda a partir de configurações promissoras."""
# Implementação simplificada de alocação de turmas em salas e horários
# Restrições: conflitos de professor e capacidade da sala
# Estrutura básica para representar salas, disciplinas e alocações
class Sala:
    # Inicializa a sala com um ID e capacidade
    def __init__(self, id, capacidade):
        # Armazena o ID da sala e sua capacidade
        self.id = id
        # Armazena os horários ocupados como um conjunto
        self.capacidade = capacidade
        # Inicializa um conjunto para rastrear horários ocupados
        self.horarios_ocupados = set()
# Estrutura para representar uma disciplina com ID, número de alunos e professor  
class Disciplina:
    # Inicializa a disciplina com ID, número de alunos e professor  
    def __init__(self, id, num_alunos, professor):
        # Armazena o ID da disciplina
        self.id = id
        # Armazena o número de alunos na disciplina
        self.num_alunos = num_alunos
        # Armazena o nome do professor responsável pela disciplina
        self.professor = professor
# Estrutura para gerenciar alocações de disciplinas em salas e horários        
class Alocacao:
    # Inicializa a alocação com listas de salas e disciplinas
    def __init__(self, salas, disciplinas):
        # Armazena as salas e disciplinas
        self.salas = salas
        # Armazena as disciplinas
        self.disciplinas = disciplinas
        # Dicionário para rastrear alocações (disciplina_id -> (sala_id, horario))
        self.alocacoes = {}
        # Dicionário para rastrear professores ocupados por horário
        self.professores_ocupados = {}
    # Função para verificar se uma disciplina pode ser alocada em uma sala e horário
    def pode_alocar(self, disciplina, sala, horario):
        # Verifica se a sala tem capacidade suficiente
        if sala.capacidade < disciplina.num_alunos:
            return False
        # Verifica se o horário está ocupado na sala
        if horario in sala.horarios_ocupados:
            return False
        # Verifica se o professor já está ocupado nesse horário
        if disciplina.professor in self.professores_ocupados.get(horario, set()):
            return False
        return True
    # Função para alocar uma disciplina em uma sala e horário
    def alocar(self, disciplina, sala, horario):
        # Verifica se a alocação é possível
        if self.pode_alocar(disciplina, sala, horario):
            # Realiza a alocação
            self.alocacoes[disciplina.id] = (sala.id, horario)
            # Marca o horário como ocupado na sala
            sala.horarios_ocupados.add(horario)
            # Marca o professor como ocupado nesse horário
            if horario not in self.professores_ocupados:
                self.professores_ocupados[horario] = set()
            # Adiciona o professor ao conjunto de ocupados nesse horário
            self.professores_ocupados[horario].add(disciplina.professor)
            return True
        return False
    #Função para desalocar uma disciplina
    def desalocar(self, disciplina, sala, horario):
        # Verifica se a disciplina está alocada
        if disciplina.id in self.alocacoes:
            # Remove a alocação
            del self.alocacoes[disciplina.id]
            # Libera o horário na sala e do professor
            sala.horarios_ocupados.remove(horario)
            # Remove o professor do horário ocupado
            self.professores_ocupados[horario].remove(disciplina.professor)
            # Se não houver mais professores ocupados nesse horário, remove a entrada
            if not self.professores_ocupados[horario]:
                # Remove o horário do dicionário se estiver vazio
                del self.professores_ocupados[horario]
            # Retorna True indicando que a desalocação foi bem-sucedida
            return True
        return False
    
# Exemplo de uso
salas = [Sala("Sala1", 30), Sala("Sala2", 50)]
disciplinas = [Disciplina("Math", 25, "ProfA"), Disciplina("Physics", 40, "ProfB")]
alocacao = Alocacao(salas, disciplinas)
# Tentar alocar disciplinas
alocacao.alocar(disciplinas[0], salas[0], "08:00- 10:00")
alocacao.alocar(disciplinas[1], salas[1], "10:00 - 12:00")
print(alocacao.alocacoes)   

{'Math': ('Sala1', '08:00- 10:00'), 'Physics': ('Sala2', '10:00 - 12:00')}
